In [1]:
import pandas as pd
from sklearn.datasets import make_classification
X,y=make_classification(n_samples=10000, n_features=2, n_redundant=0, n_clusters_per_class=1,
weights=[0.99], flip_y=0, random_state=1)
dfX=pd.DataFrame(X, columns=['a','b'])
dfy=pd.DataFrame(y, columns=['y'])
df=pd.concat([dfX,dfy],axis=1)
df

,a,b,y
0,0.222014,0.540207,0
1,1.347439,1.412824,0
2,0.537238,0.372730,0
3,2.134462,1.404819,0
4,2.315827,1.356858,0
...,...,...,...
9995,2.440385,1.695643,0
9996,-0.790502,0.194243,0
9997,1.878130,0.829500,0
9998,2.585933,1.927995,0


In [2]:
# 불균형 데이터
X1=df[['a','b']]
y1=df['y']
df['y'].value_counts()

0    9900
1     100
Name: y, dtype: int64

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X1, y1, test_size=0.2, stratify=y1, random_state=10)

model1=LogisticRegression(random_state=42)
model1.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [4]:
print("학습용 :",model1.score(X_train, y_train))
print("검증용 :",model1.score(X_test, y_test))

학습용 : 0.994125
검증용 : 0.995


In [5]:
from sklearn.metrics import confusion_matrix

pred1=model1.predict(X_test)
cm=confusion_matrix(y_test, pred1)
cm

array([[1980,    0],
       [  10,   10]], dtype=int64)

In [6]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred1))
# accuracy는 높으나 소수 클래스의 재현율(recall)이 0.5로 낮아지는 문제 발생

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1980
           1       1.00      0.50      0.67        20

    accuracy                           0.99      2000
   macro avg       1.00      0.75      0.83      2000
weighted avg       1.00      0.99      0.99      2000



In [7]:
# 균형 데이터
X,y=make_classification(n_samples=10000, n_features=2, n_redundant=0, n_clusters_per_class=1,
flip_y=0, random_state=1)
dfX=pd.DataFrame(X, columns=['a','b'])
dfy=pd.DataFrame(y, columns=['y'])
df2=pd.concat([dfX,dfy],axis=1)
df2

,a,b,y
0,0.222014,0.540207,0
1,-1.461718,1.556713,1
2,-0.185450,0.170503,1
3,-0.076351,1.745782,1
4,2.315827,1.356858,0
...,...,...,...
9995,-0.295564,2.153372,1
9996,-2.027697,-0.366690,1
9997,0.962237,1.027313,1
9998,-0.642143,2.454120,1


In [8]:
X1=df2[['a','b']]
y1=df2['y']
df2['y'].value_counts()

0    5000
1    5000
Name: y, dtype: int64

In [9]:
X_train, X_test, y_train, y_test=train_test_split(X1, y1, test_size=0.2, stratify=y1, random_state=10)

model2=LogisticRegression(random_state=42)
model2.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [10]:
print("학습용 :",model2.score(X_train, y_train))
print("검증용 :",model2.score(X_test, y_test))

학습용 : 0.896125
검증용 : 0.891


In [11]:
pred2=model2.predict(X_test)
print(classification_report(y_test,pred2)) # 정확도와 재현율이 비슷하게 처리됨

              precision    recall  f1-score   support

           0       0.88      0.91      0.89      1000
           1       0.90      0.87      0.89      1000

    accuracy                           0.89      2000
   macro avg       0.89      0.89      0.89      2000
weighted avg       0.89      0.89      0.89      2000



In [12]:
len(X1),len(y1),len(pred1)

(10000, 10000, 2000)

In [13]:
# 비대칭 데이터는 언더샘플링, 오버샘플링, 복합샘플링 등의 방법으로 데이터 비율을 맞추면 정밀도가 향상된다.
X,y=make_classification(n_samples=10000, n_features=2, n_redundant=0, n_clusters_per_class=1,
weights=[0.99], flip_y=0, random_state=1)

In [19]:
# 언더샘플링 : 데이터의 손실이 크고 중요한 특성을 가진 데이터를 잃을 수 있음
# 1. 무작위로 다수 클래스의 데이터를 없애는 단순 샘플링
from imblearn.under_sampling import RandomUnderSampler

X_sample, y_sample=RandomUnderSampler(random_state=0).fit_resample(X,y)

X_samp=pd.DataFrame(data=X_sample,columns=['a','b'])
y_samp=pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()

0    100
1    100
Name: y, dtype: int64

In [24]:
# 토멕 링크(Tomek's link) : 서로 다른 클래스에 속하는 한 쌍의 데이터
# 2. 토멕 링크 중에서 다수 클래스에 속한 샘플을 제거함으로써 데이터의 균형을 맞추는 방법
# sampling_strategy='majority' : 다수 클래스의 샘플을 제거
# sampling_strategy='not minority' : 소수 클래스를 제외하고 샘플링
# sampling_strategy='not majority' : 다수 클래스를 제외하고 샘플링
# sampling_strategy='all' : 모든 클래스를 샘플링
# sampling_strategy='auto' : 'not minority'와 같음 (기본값)
from imblearn.under_sampling import TomekLinks

X_sample, y_sample=TomekLinks(sampling_strategy='majority').fit_resample(X,y)

X_samp=pd.DataFrame(data=X_sample,columns=['a','b'])
y_samp=pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()
# 토멕 링크 중에서 다수 클래스의 샘플들을 제거하는 방식, 1:1로 맞추는 방식은 아님

0    9874
1     100
Name: y, dtype: int64

In [25]:
# CNN(Condensed Nearest Neighbour) : 1-NN 모형으로 분류되지 않는 데이터만 남기는 방법
# 3. 다수의 데이터 중에서 하나를 골라서 최근접 이웃이 다수 클래스이면 그 샘플을 빼는 방식
from imblearn.under_sampling import CondensedNearestNeighbour

X_sample, y_sample=CondensedNearestNeighbour(random_state=0).fit_resample(X,y)

X_samp=pd.DataFrame(data=X_sample,columns=['a','b'])
y_samp=pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()
# 실행 시간이 오래 걸리고, 1:1로 맞춰지지 않음

0    187
1    100
Name: y, dtype: int64

In [33]:
# One Sided Selection
# 토멕 링크 방법과 Condensed Nearest Neighbour 방법을 섞은 방식
# 4. 토멕 링크 중 다수 클래스의 샘플을 제거하고 나머지 데이터 중에서도 서로 붙어있는 다수 클래스 데이터는
#   1-NN 방법으로 제외하는 방식
from imblearn.under_sampling import OneSidedSelection

X_sample, y_sample=OneSidedSelection(random_state=0).fit_resample(X,y)

X_samp=pd.DataFrame(data=X_sample,columns=['a','b'])
y_samp=pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()

0    6593
1     100
Name: y, dtype: int64

In [34]:
# ENN(Edited Nearest Neighbours)
# 5. 다수 클래스 데이터 중 소수 클래스와 가장 가까운 (n_neighbours)개의 데이터가
#   모두 또는 다수 클래스가 아니면 삭제하는 방법. 소수 클래스 주변의 다수 클래스 데이터는 삭제됨
# kind_sel='all' : 모두
# kind_sel='mode' : 다수
from imblearn.under_sampling import EditedNearestNeighbours

# 5개의 이웃이 모두 같은 클래스가 아니면 그 샘플을 제거함
X_sample, y_sample=EditedNearestNeighbours(kind_sel='all', n_neighbors=5).fit_resample(X,y)

X_samp=pd.DataFrame(data=X_sample,columns=['a','b'])
y_samp=pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()

0    9747
1     100
Name: y, dtype: int64

In [38]:
# Neighbourhood Cleaning Rule
# 6. CNN(Condensed Nearest Neighbour) 방법과 ENN(Edited Nearest Neighbours) 방법을 섞은 것
from imblearn.under_sampling import NeighbourhoodCleaningRule

# 5개의 이웃이 모두 같은 클래스가 아니면 그 샘플을 제거함
X_sample, y_sample=NeighbourhoodCleaningRule(kind_sel='all', n_neighbors=5).fit_resample(X,y)

X_samp=pd.DataFrame(data=X_sample,columns=['a','b'])
y_samp=pd.DataFrame(data=y_sample,columns=['y'])
y_samp.y.value_counts()

0    9721
1     100
Name: y, dtype: int64

In [39]:
X_train, X_test, y_train, y_test=train_test_split(X_samp, y_samp, test_size=0.2, stratify=y_samp, random_state=10)

model3=LogisticRegression(random_state=42)
model3.fit(X_train, y_train)

D:\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression(random_state=42)

In [40]:
print("학습용 :",model3.score(X_train, y_train))
print("검증용 :",model3.score(X_test, y_test))

학습용 : 0.9950356415478615
검증용 : 0.9969465648854962


In [41]:
pred3=model3.predict(X_test)
print(classification_report(y_test, pred3))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1945
           1       1.00      0.70      0.82        20

    accuracy                           1.00      1965
   macro avg       1.00      0.85      0.91      1965
weighted avg       1.00      1.00      1.00      1965

